# Preparing the BRFSS Data

The following cell downloads the data from https://www.cdc.gov/brfss/annual_data/annual_2020.html

Codebook: https://www.cdc.gov/brfss/annual_data/2020/pdf/codebook20_llcp-v2-508.pdf

In [1]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
        
download("https://www.cdc.gov/brfss/annual_data/2020/files/LLCP2020ASC.zip")

Downloaded LLCP2020ASC.zip


The following cells download the metadata, including the column numbers where each field starts and ends.

In [2]:
import pandas as pd
import numpy as np

In [3]:
url = 'https://www.cdc.gov/brfss/annual_data/2020/llcp_varlayout_20_onecolumn.html'
tables = pd.read_html(url)
column_info = tables[0]
column_info.index = column_info['Variable Name']

In [4]:
column_info['Starting Column'] -= 1
column_info['Ending Column'] = (column_info['Starting Column'] + 
                                column_info['Field Length'])
column_info.head()

,Starting Column,Variable Name,Field Length,Ending Column
Variable Name,,,,
_STATE,0,_STATE,2,2
FMONTH,16,FMONTH,2,18
IDATE,18,IDATE,8,26
IMONTH,18,IMONTH,2,20
IDAY,20,IDAY,2,22


Here are the columns we'll use.

In [5]:
names = ['SEQNO', 'HTM4', 'WTKG3', '_BMI5', '_SEX', '_AGEG5YR', 
         '_INCOMG', '_LLCPWT']

In [6]:
cols = ['Starting Column', 'Ending Column']
colspecs_df = column_info.loc[names, cols]
colspecs_df

,Starting Column,Ending Column
Variable Name,,
SEQNO,35,45
HTM4,1988,1991
WTKG3,1991,1996
_BMI5,1996,2000
_SEX,1978,1979
_AGEG5YR,1979,1981
_INCOMG,2004,2005
_LLCPWT,1750,1760


In [7]:
colspecs = list(colspecs_df.itertuples(index=False, name=None))
colspecs

[(35, 45),
 (1988, 1991),
 (1991, 1996),
 (1996, 2000),
 (1978, 1979),
 (1979, 1981),
 (2004, 2005),
 (1750, 1760)]

Now we can read the data.

In [8]:
from zipfile import ZipFile

data_file = 'LLCP2020ASC.zip'
with ZipFile(data_file) as myzip:
    print(myzip.filelist)

[<ZipInfo filename='LLCP2020.ASC ' compress_type=deflate file_size=819592362 compress_size=43547409>]


In [9]:
with ZipFile(data_file) as myzip:
    with myzip.open('LLCP2020.ASC ') as myfile:
        brfss = pd.read_fwf(myfile, names=names, colspecs=colspecs)

In [12]:
assert brfss.shape == (401958, 8)

In [13]:
brfss.head()

,SEQNO,HTM4,WTKG3,_BMI5,_SEX,_AGEG5YR,_INCOMG,_LLCPWT
0,2020000001,170.0,4808.0,1660.0,2,8,1,284.335672
1,2020000002,163.0,7711.0,2918.0,2,10,9,171.283329
2,2020000003,173.0,NaN,NaN,2,10,5,1334.368860
3,2020000004,NaN,NaN,NaN,2,13,9,1297.486620
4,2020000005,168.0,5715.0,2034.0,2,13,9,454.815127


Height is in cm

In [14]:
brfss['HTM4'].describe()

count    380227.000000
mean        170.124260
std          10.723507
min          91.000000
25%         163.000000
50%         170.000000
75%         178.000000
max         241.000000
Name: HTM4, dtype: float64

Divide weight by 100 to get kg

In [15]:
brfss['WTKG3'] /= 100
brfss['WTKG3'].describe()

count    367010.000000
mean         82.320386
std          20.994057
min          22.680000
25%          68.040000
50%          79.380000
75%          92.990000
max         290.300000
Name: WTKG3, dtype: float64

Same with BMI

In [16]:
brfss['_BMI5'] /= 100
brfss['_BMI5'].describe()

count    360601.000000
mean         28.306313
std           6.381649
min          12.020000
25%          23.990000
50%          27.320000
75%          31.380000
max          98.430000
Name: _BMI5, dtype: float64

To map from age groups to ages, I use the following formula, which maps each code to the approximate midpoint of its range.
See https://www.cdc.gov/brfss/annual_data/2019/pdf/codebook19_llcp-v2-508.HTML

In [17]:
df = pd.DataFrame(index=range(1,14))
df['AGE'] = df.index * 5 + 17
df

,AGE
1,22
2,27
3,32
4,37
5,42
6,47
7,52
8,57
9,62
10,67


In [18]:
brfss['_AGEG5YR'].replace(14, np.nan, inplace=True)
brfss['AGE'] = brfss['_AGEG5YR'] * 5 + 17

brfss['AGE'].value_counts().sort_index()

22.0    25648
27.0    20911
32.0    23408
37.0    25492
42.0    25729
47.0    26428
52.0    31110
57.0    36219
62.0    41151
67.0    41570
72.0    37972
77.0    26607
82.0    31456
Name: AGE, dtype: int64

Write the results to an HDF file.

In [19]:
!rm brfss.hdf

rm: cannot remove 'brfss.hdf': No such file or directory


In [20]:
brfss.to_hdf('brfss.hdf', 'brfss', complevel=3)

In [21]:
%time brfss = pd.read_hdf('brfss.hdf', 'brfss')

CPU times: user 143 ms, sys: 7.95 ms, total: 151 ms
Wall time: 149 ms


In [22]:
!ls -lh brfss.hdf

-rw-rw-r-- 1 downey downey 11M Dec  4 09:39 brfss.hdf


The BRFSS uses stratified sampling. We can use resampling to generate a random representative sample.

In [23]:
def resample_rows_weighted(df, column='finalwgt'):
    """Resamples a DataFrame using probabilities proportional to given column.

    df: DataFrame
    column: string column name to use as weights

    returns: DataFrame
    """
    weights = df[column].copy()
    weights /= sum(weights)
    indices = np.random.choice(df.index, len(df), replace=True, p=weights)
    sample = df.loc[indices]
    return sample

In [24]:
np.random.seed(18)

sample = resample_rows_weighted(brfss, '_LLCPWT')
sample.shape

(401958, 9)

Write the sample to an HDF file.

In [25]:
!rm brfss_sample.hdf

In [26]:
sample.to_hdf('brfss_sample.hdf', 'brfss', complevel=3)

In [27]:
!ls -lh brfss_sample.hdf

-rw-rw-r-- 1 downey downey 12M Dec  4 09:40 brfss_sample.hdf


In [28]:
%time sample = pd.read_hdf('brfss_sample.hdf', 'brfss')

CPU times: user 151 ms, sys: 4.12 ms, total: 155 ms
Wall time: 153 ms
